In [ ]:
import gensim
gensim.__version__

In [ ]:
!pip install konlpy

## 1. 영어 Word2Vec 만들기

In [ ]:
import re
import urllib.request
import zipfile
from lxml import etree
from nltk.tokenize import word_tokenize, sent_tokenize

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
# 훈련데이터 다운로드 (ted 강연 스크립트)
urllib.request.urlretrieve("https://raw.githubusercontent.com/ukairia777/tensorflow-nlp-tutorial/main/09.%20Word%20Embedding/dataset/ted_en-20160408.xml", filename="ted_en-20160408.xml")



*   XML 파일을 받아 text 추출
*   정규표현식 sub 모듈로 배경음 부분 제거 (괄호부분 제거)
*   문장 토큰화
*   구두점 제거, 소문자 변환
*   단어 토큰화

여기서 샘플은 문장형태임!



In [ ]:
targetXML = open('ted_en-20160408.xml', 'r', encoding='UTF8')
target_text = etree.parse(targetXML)

# xml 파일에서 추출한 text에서 <content>와 </content> 사이의 내용만 추출하기
parse_text = '\n'.join(target_text.xpath('//content/text()'))

# 정규 표현식의 sub 모듈을 통해 content 중간에 등장하는 (Audio), (Laughter) 등의 배경음 부분을 제거.
# 해당 코드는 괄호로 구성된 내용을 제거
content_text = re.sub(r'\([^)]*\)', '', parse_text)

# 입력 텍스트에 대해서 NLTK를 이용하여 문장토큰화를 수행.
sent_text = sent_tokenize(content_text)

# 각 문장에 대해서 구두점을 제거하고, 대문자를 소문자로 변환.
normalized_text = []
for string in sent_text:
     tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
     normalized_text.append(tokens)

# 각 문장에 대해서 NLTK를 이용하여 '단어' 토큰화를 수행.
result = [word_tokenize(sentence) for sentence in normalized_text]

In [ ]:
print('총 샘플의 개수 : {}'.format(len(result)))

In [ ]:
# 샘플 문장 3개 추출해 토큰화된 '단어' 확인
for line in result[:3]:
    print(line)

In [ ]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors



*   vector_size : 임베딩된 벡터의 차원
*   window : 컨텍스트 윈도우 크기
*   min_count : 단어 최소 빈도 수 제한
*   workers : 학습을 위한 프로세스 수
*   sg = 0 은 CBOW, 1은 Skip gram

**CBOW** 는 target word 근처의 문맥을 파악해 target word 를 파악하는 방법

**Skip gram** 은 target word를 보고 문맥을 예측하는 방법!

window 인자는 근처 문맥의 단얼ㄹ 몇개로 할지, 그 크기를 결정하는 인자



In [ ]:
model = Word2Vec(sentences=result, vector_size=100, window=5, min_count=5, workers=4, sg=0)

model.wv.most_similar 을 통해 입력한 단어를 출력 가능 (코사인 유사도 기반)

In [ ]:
model_result = model.wv.most_similar("dance")
print(model_result)

유사도 기반으로 산정된 벡터이기 때문에 연산 가능

In [ ]:
model.wv.most_similar(positive = ["woman",], negative = ["man"])

In [ ]:
model.wv.save_word2vec_format('eng_w2v') # 모델 저장
loaded_model = KeyedVectors.load_word2vec_format("eng_w2v") # 모델 불러오기

In [ ]:
model_result = loaded_model.most_similar("dance")
print(model_result)

Skip gram 으로 해보기

In [ ]:
model_s = Word2Vec(sentences=result, vector_size=100, window=5, min_count=5, workers=4, sg=1)

In [ ]:
model_result_s = model_s.wv.most_similar("dance")
print(model_result_s)

In [ ]:
model_s.wv.most_similar(positive = ["woman",], negative = ["man"])

## 한국어 Word2Vec 만들기

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import urllib.request
from gensim.models.word2vec import Word2Vec
from konlpy.tag import Okt
import tqdm

Okt 느릴경우 Mecab 사용!

아래는 설치방법

In [ ]:
!pip install mecab-ko

In [ ]:
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git

In [ ]:
cd Mecab-ko-for-Google-Colab

In [ ]:
!bash install_mecab-ko_on_colab_light_220429.sh
from konlpy.tag import Mecab

In [ ]:
# 네이버 영화 리뷰 데이터 다운로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt", filename="ratings.txt")

In [ ]:
train_data = pd.read_table('ratings.txt')

In [ ]:
train_data.head()

In [ ]:
print(len(train_data))

In [ ]:
train_data.info()

document 결측치 행 8개 발견 -> 제거

In [ ]:
train_data = train_data.dropna(how = 'any')

In [ ]:
print(len(train_data))

In [ ]:
# 정규 표현식을 통한 한글 외 문자 제거
train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")

In [ ]:
from google import colab
colab.drive.mount("/content/drive")

In [ ]:
# 불용어 리스트 정의
with open('/content/drive/MyDrive/24-winter KUBIG NLP/WEEK 1 예습과제 - text preprocessing/stopword.txt',  encoding='utf-8') as f:
    list_file = f.readlines()
    stopwords = list_file[0].split(",")

In [ ]:
# using okt
okt = Okt()

tokenized_data = []
for sentence in tqdm.tqdm(train_data['document']):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    tokenized_data.append(stopwords_removed_sentence)

In [ ]:
# using mecab

import Mecab

#mecab = mecab.Mecab()

tokenized_data = []
for sentence in tqdm.tqdm(train_data['document']):
    tokenized_sentence = mecab.morphs(sentence) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    tokenized_data.append(stopwords_removed_sentence)

In [ ]:
# 리뷰 3개의 토큰화 결과만 출력
print(tokenized_data[:3])

In [ ]:
# 리뷰 길이 분포 확인
print('리뷰의 최대 길이 :',max(len(review) for review in tokenized_data))
print('리뷰의 평균 길이 :',sum(map(len, tokenized_data))/len(tokenized_data))
plt.hist([len(review) for review in tokenized_data], bins=50)
plt.xlabel('length of samples')
plt.ylabel('freauency')
plt.show()

In [ ]:
#CBOW 로 모델 형성
model = Word2Vec(sentences = tokenized_data, vector_size = 100, window = 5, min_count = 5, workers = 4, sg = 0)

In [ ]:
model.wv.vectors.shape

In [ ]:
print(model.wv.most_similar("이동진"))

In [ ]:
model.wv.most_similar(positive=['타짜'])

In [ ]:
model.wv.most_similar(positive=['송강호'], negative=['주연'])

In [ ]:
model.wv.similarity('송강호', '하정우')

In [ ]:
model.wv.similarity('송강호', '변호')